![image](https://docs.google.com/uc?export=download&id=15DUc09hFGqR8qcpYiN1OajRNaASmiL6d)

# **Scikit-image Final Bootcamp**

# **Contenido**
## **Extracción de Características**
-  Ya segmentamos nuestras imágenes, y ¿Ahora qué?
- Buscamos formas de representar y/o describir lo que encontramos en términos de sus bordes, de su región o de la imagen completa


1. [**Preprocesamiento de Contornos**](#id1):
    - [**Seguimiento de Contornos Vs. Detección de Contornos**](#id1-1)
    - [**Esqueletización Morfológica Vs. Transformada del Eje Medio**](#id1-2)
    - [**Código de Cadena de Freeman**](#id1-3)
    - [**Firma de Contornos**](#id1-4)
    - [**Polígonos de Perímetro Mínimo**](#id1-5)
2. [**Características de Frontera**](#id2):
    - [**Diámetro Local**](#id2-1)
    - [**Longitud**](#id2-2)
3. [**Características de Región**](#id3):
    - [**Circularidad**](#id3)
    - [**Excentricidad**](#id3)
    - [**Compacidad**](#id3)
    - [**Número de Euler**](#id3)
    - [**Descriptores de Textura - Momentos Estadísticos**](#id3-1)
    - [**Momentos de Imagen**](#id3-2)
4. [**Características de la Imagen**](#id4):
    - [**Histograma De Gradientes Orientados**](#id4-1)

In [ ]:
import cv2
from skimage import io
from utils import visualization as vis
from skimage.filters import threshold_otsu, sobel, gaussian
from skimage.exposure import equalize_hist, rescale_intensity
from skimage.measure import (find_contours, 
                             approximate_polygon, 
                             subdivide_polygon, label, 
                             regionprops, regionprops_table)
from skimage import measure
from skimage.feature import hog
from skimage import data
from skimage import morphology

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from skimage import color

import scipy.ndimage as ndi
from scipy import stats

In [ ]:
url = "https://github.com/PacktPublishing/Python-Image-Processing-Cookbook/blob/master/Chapter%2004/images/planets.png?raw=true"

# **Preprocesamiento de Contornos**<a name="id1"></a>
## **Seguimiento de Contornos Vs. Detección de Contornos**<a name="id1-1"></a>
![image](https://i.stack.imgur.com/RNImW.jpg)

In [ ]:
img = vis.download_content(url)

In [ ]:
vis.imshow(img)

In [ ]:
img_gray = color.rgb2gray(img)

In [ ]:
vis.imshow(img_gray)

In [ ]:
thresh = threshold_otsu(img_gray)

In [ ]:
segmented_img = (img_gray > thresh).astype("uint8")

In [ ]:
vis.imshow(segmented_img)

In [ ]:
contours, _ = cv2.findContours(segmented_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
contours

In [ ]:
draw = cv2.drawContours(segmented_img.copy(), contours, -1, (255, 255, 0), 2)

In [ ]:
vis.imshow(draw)

In [ ]:
edges = sobel(segmented_img)
vis.imshow(edges)

In [ ]:
vis.visualize_subplot([draw, edges], 
                      ["Seguimiento de Contornos", 
                       "Detección de Contornos con Sobel"], (1, 2), (12, 6))

### **Preguntas**
- ¿Cuál es la diferencia entre la detección de contornos y el seguimiento de contornos?
- ¿Cómo se podría preprocesar la imagen para recuperar la forma circular de los planetas, que se perdió durante la umbralización, sin afectar los anillos de Saturno?

## **Esqueletización Morfológica Vs. Transformada del Eje Medio**<a name="id1-2"></a>

In [ ]:
url_angio = "https://www.radiologyinfo.org/gallery-items/images/ICA-Angiogram.jpg"

In [ ]:
img = vis.download_content(url_angio)

In [ ]:
img_gray = color.rgb2gray(img)

In [ ]:
vis.imshow(img_gray)

In [ ]:
segmented = (img_gray < 0.4).astype("uint8")

In [ ]:
vis.imshow(segmented)

In [ ]:
skeleton = vis.skeletonize(segmented)

In [ ]:
vis.imshow(skeleton)

In [ ]:
medial_transform = morphology.medial_axis(segmented)

In [ ]:
vis.imshow(medial_transform)

In [ ]:
vis.visualize_subplot([skeleton, medial_transform],
                      ["Esqueletización Morfológica", 
                       "Transformada del Eje Medio"], (1, 2), (18, 9))

In [ ]:
segmented = np.logical_not(data.horse()).astype("uint8")

In [ ]:
vis.imshow(segmented)

In [ ]:
skeleton = vis.skeletonize(segmented)
medial_transform = morphology.medial_axis(segmented)

In [ ]:
vis.visualize_subplot([skeleton, medial_transform],
                      ["Esqueletización Morfológica", 
                       "Transformada del Eje Medio"], (1, 2), (18, 9))

In [ ]:
url_tree = "https://estaticos.muyinteresante.es/media/cache/1000x460_thumb/uploads/images/test/5af18b675bafe8cd7d3c986b/arbol.jpg"

In [ ]:
arbol = vis.download_content(url_tree)

In [ ]:
arbol_gray = color.rgb2gray(arbol)

In [ ]:
vis.imshow(arbol_gray)

In [ ]:
thresh = threshold_otsu(arbol_gray)
segmented = (arbol_gray < thresh).astype("uint8")

In [ ]:
vis.imshow(segmented)

In [ ]:
medial_transform = morphology.medial_axis(segmented)

In [ ]:
skeleton = vis.skeletonize(segmented)

In [ ]:
vis.visualize_subplot([skeleton, medial_transform],
                      ["Esqueletización Morfológica", 
                       "Transformada del Eje Medio"], (1, 2), (18, 9))

### **Preguntas**
- Repetir el procedimiento con la siguiente [url](https://image.shutterstock.com/image-photo/group-people-260nw-602783837.jpg).

## **Código de Cadena de Freeman**<a name="id1-3"></a>
- ¿Cómo describimos una imagen en términos de su contorno? Aquí es donde entramos a usar el código de la cadema de Freeman.
![image](https://www.researchgate.net/profile/Kabir_Mamun/publication/305791915/figure/fig4/AS:391040709218307@1470242778549/Freeman-chain-code-of-a-contour.png)

In [ ]:
leaf_url = "https://image.shutterstock.com/image-photo/green-perilla-leaf-known-shiso-260nw-416872249.jpg"

In [ ]:
leaf = vis.download_content(leaf_url)
leaf_gray = color.rgb2gray(leaf)

In [ ]:
vis.imshow(leaf_gray)

In [ ]:
segmented = (leaf_gray < .9).astype("uint8")
segmented = segmented[:250, :]

In [ ]:
vis.imshow(segmented)

In [ ]:
y, x = vis.find_start_point(segmented)

In [ ]:
vis.imshow(segmented)
plt.scatter(x, y, color="r")
plt.show()

In [ ]:
border, chain, count = vis.find_contours(segmented)

In [ ]:
contours = find_contours(segmented, .8)

In [ ]:
contour = max(contours, key=len)

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(121)
plt.plot(border[..., 1], border[..., 0], linewidth=2, color="r")
vis.imshow(leaf_gray, "Freeman Chain Code")

plt.subplot(122)
plt.plot(contour[..., 1], contour[..., 0], linewidth=2, color="b")
vis.imshow(leaf_gray, "Scikit-Image Contour")

plt.show()

In [ ]:
for dir, code in zip(border, chain):
    print(f"Coordenada x: {dir[1]}, y: {dir[0]} - Freeman Code: {code}")

## **Firma de Contornos**<a name="id1-4"></a>

In [ ]:
cy, cx = ndi.center_of_mass(leaf_gray)

In [ ]:
plt.scatter(cx, cy, color="r")
vis.imshow(leaf_gray)

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(border[..., 1], border[..., 0])
plt.scatter(cx, cy)
plt.grid(linestyle="--")
plt.title("Non-centered Image")
plt.ylim([250, 0])
plt.show()

In [ ]:
border[..., 1] -= int(cx)
border[..., 0] -= int(cy)

In [ ]:
polar_contour = np.array([vis.cart2pol(x, y) for x, y in border])

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(border[..., 1], border[..., 0])
plt.scatter(0, 0)
plt.grid(linestyle="--")
plt.title("Zero-centered Image")
plt.ylim([110, -130])
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.scatter(polar_contour[..., 1], polar_contour[..., 0], linewidth=0, s=3)
plt.title("Polar Coordinates")
plt.grid(linestyle="--")

plt.subplot(122)
plt.scatter(border[..., 1], border[..., 0], s=1)
plt.grid(linestyle="--")
plt.ylim([110, -130])
plt.title("Cartesian Coordinates")
plt.show()

## **Polígonos de Perímetro Mínimo**<a name="id1-5"></a>

In [ ]:
plt.figure(figsize=(18, 12))

plt.subplot(231)
coords = approximate_polygon(border, tolerance=1)
plt.plot(coords[..., 1], coords[..., 0])
plt.grid(linestyle="--")
plt.title("Tolerance 1")

plt.subplot(232)
coords = approximate_polygon(border, tolerance=2)
plt.plot(coords[..., 1], coords[..., 0])
plt.grid(linestyle="--")
plt.title("Tolerance 2")

plt.subplot(233)
coords = approximate_polygon(border, tolerance=4)
plt.plot(coords[..., 1], coords[..., 0])
plt.grid(linestyle="--")
plt.title("Tolerance 4")

plt.subplot(234)
coords = approximate_polygon(border, tolerance=6)
plt.plot(coords[..., 1], coords[..., 0])
plt.grid(linestyle="--")
plt.title("Tolerance 6")

plt.subplot(235)
coords = approximate_polygon(border, tolerance=8)
plt.plot(coords[..., 1], coords[..., 0])
plt.grid(linestyle="--")
plt.title("Tolerance 8")

plt.subplot(236)
coords = approximate_polygon(border, tolerance=10)
plt.plot(coords[..., 1], coords[..., 0])
plt.grid(linestyle="--")
plt.title("Tolerance 10")

plt.show()

In [ ]:
plt.figure(figsize=(18, 12))

plt.subplot(231)
coords = approximate_polygon(border, tolerance=1)
polar = np.array([vis.cart2pol(x, y) for x, y in coords])
plt.plot(polar[:-15, 1], polar[:-15, 0])
plt.grid(linestyle="--")
plt.title("Tolerance 1")

plt.subplot(232)
coords = approximate_polygon(border, tolerance=2)
polar = np.array([vis.cart2pol(x, y) for x, y in coords])
plt.plot(polar[:-15, 1], polar[:-15, 0])
plt.grid(linestyle="--")
plt.title("Tolerance 2")

plt.subplot(233)
coords = approximate_polygon(border, tolerance=4)
polar = np.array([vis.cart2pol(x, y) for x, y in coords])
plt.plot(polar[:-15, 1], polar[:-15, 0])
plt.grid(linestyle="--")
plt.title("Tolerance 4")

plt.subplot(234)
coords = approximate_polygon(border, tolerance=6)
polar = np.array([vis.cart2pol(x, y) for x, y in coords])
plt.plot(polar[:-7, 1], polar[:-7, 0])
plt.grid(linestyle="--")
plt.title("Tolerance 6")

plt.subplot(235)
coords = approximate_polygon(border, tolerance=8)
polar = np.array([vis.cart2pol(x, y) for x, y in coords])
plt.plot(polar[:-5, 1], polar[:-5, 0])
plt.grid(linestyle="--")
plt.title("Tolerance 8")

plt.subplot(236)
coords = approximate_polygon(border, tolerance=10)
polar = np.array([vis.cart2pol(x, y) for x, y in coords])
plt.plot(polar[:-3, 1], polar[:-3, 0])
plt.grid(linestyle="--")
plt.title("Tolerance 10")

plt.show()

### **Preguntas**
- ¿Qué pasaría si no uso el centro de masa como referencia para centrar la imagen?
- ¿Cómo afecta la toleracia al polígono de perímetro mínimo?
- Repetir el ejercicio con las siguiente [url](https://d25rq8gxcq0p71.cloudfront.net/dictionary-images/324/hand_FD9zpJx.jpg)

In [ ]:
url_planets = "https://hipertextual.com/files/2014/02/sistema-solar-tierra.jpg"

In [ ]:
planets = vis.download_content(url_planets)
planets_gray = color.rgb2gray(planets)

In [ ]:
vis.imshow(planets_gray)

In [ ]:
segmented = (planets_gray > 0.3).astype("uint8")

In [ ]:
vis.imshow(segmented)

In [ ]:
contours = find_contours(segmented, .8)

In [ ]:
contour = max(contours, key=len)

In [ ]:
plt.scatter(contour[..., 1], contour[..., 0], s=1)
vis.imshow(segmented)

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(contour[... ,1], contour[..., 0], label=f"Original Contour")
for i in range(5, 21, 5):
    coords = approximate_polygon(contour, tolerance=i)
    plt.plot(coords[..., 1], coords[..., 0], label=f"$Tolerance={i}$")
plt.grid(linestyle="--")
plt.legend(loc="best")
plt.show()

### **Preguntas**
- Repetir el ejercicio con la siguiente [url](https://i.blogs.es/981b0d/screenshot-2/1366_2000.jpg) y suprima los astros pequeños.

# **Características de Frontera**<a name="id2"></a>
- Diametro Local
$$diameter(B)=max_{i, j}[D(p_i, p_j)]$$

- Longitud
$$length_m=[(x_2 - x_1)^2 + (y_2 - y_1)^2]^{\frac{1}{2}}$$

## **Diámetro Local**<a name="id2-1"></a>

In [ ]:
leaf_url = "https://image.shutterstock.com/image-photo/green-perilla-leaf-known-shiso-260nw-416872249.jpg"

In [ ]:
leaf = vis.download_content(leaf_url)
leaf_gray = color.rgb2gray(leaf)

In [ ]:
segmented = (leaf_gray < .9).astype("uint8")
segmented = segmented[:250, :]

In [ ]:
vis.imshow(segmented)

In [ ]:
border, _, _ = vis.find_contours(segmented)

In [ ]:
plt.plot(border[..., 1], border[..., 0])
plt.show()

In [ ]:
a, b, dist = vis.diameter(border)

In [ ]:
plt.plot(border[..., 1], border[..., 0])
plt.scatter(a[1], a[0], c="b", marker="s")
plt.scatter(b[1], b[0], c="b", marker="s")

plt.plot((a[1], b[1]), (a[0], b[0]), "--b")
vis.imshow(segmented)
plt.show()

## **Longitud**<a name="id2-2"></a>

In [ ]:
perimeter = vis.perimeter(border)

In [ ]:
perimeter

In [ ]:
regionprops(segmented)[0]["perimeter"]

# **Características de Región**<a name="id3"></a>
1. **Circularidad:** $\frac{4\pi A}{p^2}$

2. **Compacidad:** $\frac{p^2}{A}$

3. **Excentricidad:** $\frac{c}{a}=\frac{\sqrt{a^2 - b^2}}{a}=\sqrt{1-(b/a)^2}$

4. **Número de Euler:** $C-H$

In [ ]:
url_elipse = "https://p4.wallpaperbetter.com/wallpaper/5/875/596/nature-hole-texture-black-wallpaper-preview.jpg"

In [ ]:
elipse = vis.download_content(url_elipse)
elipse_gray = color.rgb2gray(elipse)

In [ ]:
vis.imshow(elipse_gray)

In [ ]:
segmented = (elipse_gray > .3).astype("uint8")
segmented = segmented[210:330, 250:480]

vis.imshow(segmented)

In [ ]:
label_elipse = label(segmented)
vis.imshow(label_elipse)

In [ ]:
regions = regionprops(label_elipse)

In [ ]:
# Graficar Ejes.
for props in regions:
    if props["area"] > 500:
        y0, x0 = props["centroid"]
        
        # Cálculo de Números
        circ = 4*np.pi*props["area"]/props["perimeter"]
        comp = props["perimeter"]**2 / props["area"]
        label_v = f"Perimeter={props['perimeter']:.2f}\n"
        label_v += f"Area={props['area']}\n"
        label_v += f"Eccentricity={props['eccentricity']:.2f}\n"
        label_v += f"Circularity={circ:.2f}\n"
        label_v += f"Compactness={comp:.2f}\n"
        label_v += f"Euler Number={props['euler_number']}"
        plt.scatter(x0, y0, label=label_v, c="b", marker="s")
        
        # Bounding box
        minr, minc, maxr, maxc = props["bbox"]
        bx = (minc, maxc, maxc, minc, minc)
        by = (minr, minr, maxr, maxr, minr)
        plt.plot(bx, by, "--g", linewidth=2.5)
        
        # Vertices
        orientation = props["orientation"]
        x1 = x0 + np.cos(orientation) * .5 * props["minor_axis_length"]
        y1 = y0 - np.sin(orientation) * .5 * props["minor_axis_length"]
        
        x2 = x0 - np.sin(orientation) * .5 * props["major_axis_length"]
        y2 = y0 - np.cos(orientation) * .5 * props["major_axis_length"]
        
        plt.plot((x0, x1), (y0, y1), '-vb', linewidth=2)
        plt.plot((x0, x2), (y0, y2), '-vb', linewidth=2)
        
vis.imshow(label_elipse, "Labeled Image")
plt.legend(bbox_to_anchor=(1, 1.04), fontsize=11)
plt.show()

In [ ]:
props_table = regionprops_table(label_elipse, properties=["area",
                                                          "eccentricity", 
                                                          "perimeter", 
                                                          "euler_number"])

In [ ]:
props_df = pd.DataFrame(props_table)

In [ ]:
props_df = props_df.query("area > 100")

In [ ]:
props_df.loc[:, "circularity"] = props_df["area"] * 4*np.pi / props_df["perimeter"]
props_df.loc[:, "compactness"] = props_df["perimeter"]**2 / props_df["area"]

In [ ]:
props_df

In [ ]:
props_df.describe()

In [ ]:
url_palo = "https://drive.google.com/u/0/uc?id=1gZK2xWh7SYO2phDuzthsX84M278nF3YS&export=download"
url_circle = "https://drive.google.com/u/0/uc?id=13zop5mfTsuTJ-QRPWIv-xndZ25dxfA73&export=download"

In [ ]:
img_palo = color.rgb2gray(vis.download_content(url_palo)).astype("uint8")
img_circle = color.rgb2gray(vis.download_content(url_circle)).astype("uint8")

In [ ]:
palo_table = pd.DataFrame(regionprops_table(img_palo, properties=["area",
                                                                  "eccentricity", 
                                                                  "perimeter", 
                                                                  "euler_number", 
                                                                  "inertia_tensor_eigvals"]))
circle_table = pd.DataFrame(regionprops_table(img_circle, properties=["area",
                                                                      "eccentricity", 
                                                                      "perimeter", 
                                                                      "euler_number", 
                                                                      "inertia_tensor_eigvals"]))

In [ ]:
df = pd.concat([palo_table, circle_table])
df.index = ["Stick", "Circle"]

In [ ]:
df

## **Descriptores de Textura - Momentos Estadísticos** <a name="id3-1"></a>


In [ ]:
url_smooth = "https://st2.depositphotos.com/4327493/11109/i/950/depositphotos_111097754-stock-photo-marble-stone-texture-background-smooth.jpg"

In [ ]:
brick = vis.scale(data.brick(), 0, 255)
grass = vis.scale(data.grass(), 0, 255)
gravel = vis.scale(data.gravel(), 0, 255)
smooth = vis.scale(color.rgb2gray(vis.download_content(url_smooth)), 0, 255)

In [ ]:
vis.visualize_subplot([grass, brick, gravel, smooth], 
    ["Grass", "Brick", "Gravel", "Smooth"], 
    (1, 4), (24, 6)
)

In [ ]:
plt.figure(figsize=(24, 6))
plt.subplot(141)
sns.distplot(grass.ravel())
plt.title("Grass")

plt.subplot(142)
sns.distplot(brick.ravel())
plt.title("Brick")

plt.subplot(143)
sns.distplot(gravel.ravel())
plt.title("Gravel")

plt.subplot(144)
sns.distplot(smooth.ravel())
plt.title("Smooth")

plt.show();

In [ ]:
data_dict = {
    "texture": ["grass", "brick", "gravel", "smooth"],
    "mean": [grass.mean(), brick.mean(), gravel.mean(), smooth.mean()],
    "std": [grass.std(), brick.std(), gravel.std(), smooth.std()],
    "R (norm)": [vis.rugosity(grass), vis.rugosity(brick), 
                 vis.rugosity(gravel), vis.rugosity(smooth)],
    "3rd Moment": [vis.moments(grass), vis.moments(brick), 
                   vis.moments(gravel), vis.moments(smooth)],
}

In [ ]:
pd.DataFrame(data_dict)

## **Momentos de Imagen**<a name="id3-2"></a>

In [ ]:
circulo_url = "https://drive.google.com/u/0/uc?id=11-4_w2ZpQw4c1KqnFSM7veIyumrSFwF2&export=download"
palo_url = "https://drive.google.com/u/0/uc?id=1aTQn0L5SJ3yoTaRYdQZYPFI3FlAd7hGz&export=download"

In [ ]:
img_palo = color.rgb2gray(vis.download_content(palo_url))
img_circle = color.rgb2gray(vis.download_content(circulo_url))

img_palo_seg = (img_palo > 0).astype("uint8")
img_circle_seg = (img_circle > 0).astype("uint8")

In [ ]:
regions_palo_seg = regionprops(img_palo_seg)
regions_circle_seg = regionprops(img_circle_seg)

In [ ]:
regions_palo_seg[0]["inertia_tensor_eigvals"]

In [ ]:
centroid_palo = vis.centroid(img_palo)
centroid_palo_seg = vis.centroid(img_palo_seg)

centroid_circle = vis.centroid(img_circle)
centroid_circle_seg = vis.centroid(img_circle_seg)

In [ ]:
data_dict = {
    "Imagen": ["Binary Stick", "Binary Circle", "Grayscale Stick", 
               "Grayscale Circle"],
    "Area/Mass": [measure.moments(img_palo_seg, 0)[0], 
                measure.moments(img_circle_seg, 0)[0], 
                measure.moments(img_palo, 0)[0], 
                measure.moments(img_circle, 0)[0]],
    "Centroid": [centroid_palo_seg, centroid_circle_seg, 
                centroid_palo, centroid_circle],
    "Inertia Eigenvals": [regions_palo_seg[0]["inertia_tensor_eigvals"], 
                          regions_circle_seg[0]["inertia_tensor_eigvals"], 
                          regions_palo_seg[0]["inertia_tensor_eigvals"], 
                          regions_circle_seg[0]["inertia_tensor_eigvals"]]
}

In [ ]:
df = pd.DataFrame(data_dict)

In [ ]:
for col in df.columns[1:]:
    df[col] = df.apply(lambda x: np.round(x[col], 2), axis=1)

In [ ]:
df

In [ ]:
plt.figure(figsize=(12, 12))

plt.subplot(221)
vis.imshow(img_palo_seg, "Segmented Stick")
plt.scatter(centroid_palo_seg[0], centroid_palo_seg[1])

plt.subplot(222)
vis.imshow(img_palo, "Grayscale Stick")
plt.scatter(centroid_palo[0], centroid_palo[1])

plt.subplot(223)
vis.imshow(img_circle_seg, "Segmented Circle")
plt.scatter(centroid_circle_seg[0], centroid_circle_seg[1])

plt.subplot(224)
vis.imshow(img_circle, "Grayscale Circle")
plt.scatter(centroid_circle[0], centroid_circle[1])

plt.show()

# **Características de la Imagen**<a name="id4"></a>
## **Histograma De Gradientes Orientados**<a name="id4-1"></a>
![image](https://upload.wikimedia.org/wikipedia/commons/a/a1/HOG_scikit-image_AngelaMerkel.jpeg)

In [ ]:
url_bowie = "https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQD15A8V2TPDP3V615gaeVei6--F2OWSgbEzA&usqp=CAU"

In [ ]:
bowie = vis.download_content(url_bowie)
bowie_gray = color.rgb2gray(bowie)

In [ ]:
vis.imshow(bowie_gray)

In [ ]:
_, hog_image = hog(bowie_gray, orientations=8, 
                   pixels_per_cell=(16, 16), 
                   cells_per_block=(1, 1), visualize=True,
                   multichannel=False)

In [ ]:
vis.imshow(hog_image)

In [ ]:
_, hog_image = hog(bowie_gray, orientations=8, 
                   pixels_per_cell=(4, 4), 
                   cells_per_block=(1, 1), visualize=True,
                   multichannel=False)
vis.imshow(hog_image)

In [ ]:
plt.figure(figsize=(18, 9))
plt.subplot(121)
_, hog_image = hog(bowie_gray, orientations=8, 
                   pixels_per_cell=(16, 16), 
                   cells_per_block=(1, 1), visualize=True,
                   multichannel=False)
vis.imshow(hog_image, "Pixels Per Cell: (16, 16)")

plt.subplot(122)
_, hog_image = hog(bowie_gray, orientations=8, 
                   pixels_per_cell=(4, 4), 
                   cells_per_block=(1, 1), visualize=True,
                   multichannel=False)
vis.imshow(hog_image, "Pixels Per Cell: (4, 4)")
plt.show()

In [ ]:
equalized_bowie = equalize_hist(bowie_gray)

In [ ]:
vis.imshow(equalized_bowie)

In [ ]:
_, hog_image = hog(equalized_bowie, orientations=8, 
                   pixels_per_cell=(4, 4), 
                   cells_per_block=(1, 1), visualize=True,
                   multichannel=False)
hog_rescaled = vis.scale(hog_image, 0., 1., dtype="float")
vis.imshow(hog_rescaled)

In [ ]:
plt.figure(figsize=(18, 9))
plt.subplot(121)
_, hog_image = hog(bowie_gray, orientations=8, 
                   pixels_per_cell=(4, 4), 
                   cells_per_block=(1, 1), visualize=True,
                   multichannel=False)
vis.imshow(hog_image, "Non-Equalized")

plt.subplot(122)
_, hog_image = hog(equalized_bowie, orientations=8, 
                   pixels_per_cell=(4, 4), 
                   cells_per_block=(1, 1), visualize=True,
                   multichannel=False)
vis.imshow(hog_image, "Equalized")

In [ ]:
_, hog_image = hog(bowie, orientations=8, 
                   pixels_per_cell=(4, 4), 
                   cells_per_block=(1, 1), visualize=True,
                   multichannel=True)
vis.imshow(hog_image, "Colored Image HoG")

In [ ]:
equalized_bowie_color = np.zeros_like(bowie, dtype="float")
for i in range(3):
    equalized_bowie_color[..., i] = equalize_hist(bowie[..., i])
    
vis.imshow(equalized_bowie_color, "Equalized Bowie")

In [ ]:
_, hog_image = hog(equalized_bowie_color, orientations=8, 
                   pixels_per_cell=(4, 4), 
                   cells_per_block=(1, 1), visualize=True,
                   multichannel=True)
vis.imshow(hog_image)

In [ ]:
plt.figure(figsize=(18, 9))

plt.subplot(121)
_, hog_image = hog(bowie, orientations=8, 
                   pixels_per_cell=(4, 4), 
                   cells_per_block=(1, 1), visualize=True,
                   multichannel=True)
vis.imshow(hog_image, "Colored Image Non-equalized")

plt.subplot(122)
_, hog_image = hog(equalized_bowie_color, orientations=8, 
                   pixels_per_cell=(4, 4), 
                   cells_per_block=(1, 1), visualize=True,
                   multichannel=True)
vis.imshow(hog_image, "Colored Image Equalized")

### **Preguntas**
1. Considera ud que la ecualización de la imagen mejoró el HoG? ¿Por qué?
2. Qué otras técnicas pueden ser aplicadas para mejorar el HoG?
3. Qué pasa cuando variamos el parámetro `pixels_per_cell`? Qué pasa cuando varíamos el parámetro `orientations`? Igualmente, qué pasa cuando varíamos el parámetro `cell_per_block`?
4. Realice el mismo procedimiento con la siguiente [url](https://drive.google.com/u/0/uc?id=1kQnVSScmxKLzcJ1kQ_t59vUbp7SYbBzz&export=download). Realice todas las operaciones de procesamiento de imagen que considere necesarias para poder obtener un buen HoG.